In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBClassifier

from utils import split_reservations
from features import X1_cxl_cols, X2_cxl_cols

In [2]:
pd.options.display.max_rows = 60
pd.options.display.max_columns = 60

In [6]:
h1_res = pd.read_pickle("pickle/h1_res.pick")
h2_res = pd.read_pickle("pickle/h2_res.pick")
h1_dbd = pd.read_pickle("pickle/h1_dbd.pick")
h2_dbd = pd.read_pickle("pickle/h2_dbd.pick")

In [7]:
X1_train, X1_test, y1_train, y1_test = split_reservations(h1_res, "2017-08-01", features=X1_cxl_cols)

KeyError: 'CheckoutDate'

In [8]:
def regularization(X, y):
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=42)
    
    # standardize data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.values)
    X_val_scaled = scaler.transform(X_val)
    
    # logistic regression
    lr = LogisticRegression(max_iter=500)
    lr_model = lr.fit(X_train_scaled, y_train)
    lr_score = lr_model.score(X_val_scaled, y_val)
    
    # ridge
    ridge = RidgeCV(cv=5)
    ridge_model = ridge.fit(X_train_scaled, y_train)
    ridge_score = ridge_model.score(X_val_scaled, y_val)
    
    # lasso
    lasso = LassoCV(cv=5)
    lasso_model = lasso.fit(X_train_scaled, y_train)
    lasso_score = lasso_model.score(X_val_scaled, y_val)
    
    print('\nSIMPLE LR Validation R^2 score was:', lr_score)
#     print('Feature coefficient results: \n')
    print('Test score: ', lr_model.score(X_test, y_test))
#     for feature, coef in zip(X.columns, lr_model.coef_):
#         print(feature, ':', f'{coef:.2f}') 
    
    print('\nRIDGE Validation R^2 score was:', ridge_score)
#     print('Feature coefficient results: \n')
#     for feature, coef in zip(X.columns, ridge_model.coef_):
#         print(feature, ':', f'{coef:.2f}') 
        
    print('\nLASSO Validation R^2 score was:', lasso_score)
#     print('Feature coefficient results: \n')
#     for feature, coef in zip(X.columns, lasso_model.coef_):
#         print(feature, ':', f'{coef:.2f}') 
    

regularization(X1, y1)
    
    


SIMPLE LR Validation R^2 score was: 0.8300049925112332
Test score:  0.2829505741387918

RIDGE Validation R^2 score was: 0.3150907949370658

LASSO Validation R^2 score was: 0.31497972457165135


## XGBoost (Hotel 1): Hyperparameter Tuning with Grid Search

In [6]:
estimator = XGBClassifier(objective='binary:logistic',
                          use_label_encoder=False,
                          eval_metric='logloss',
                          random_state=42,
                          max_depth=5
)

params = {
    'learning_rate': [0.08, 0.09, 0.1, 0.11, 0.12],
    'n_estimators': [380, 390, 400, 410, 420, 475]
}

grid_search_1 = GridSearchCV(
    estimator = estimator,
    param_grid = params,
    n_jobs=-1,
    verbose=100
)

In [7]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2,random_state=42)

In [8]:
%%time
grid_xgb_fit_1 = grid_search_1.fit(X1_train, y1_train)
print("The best parameters are: \n", grid_xgb_fit_1.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
The best parameters are: 
 {'learning_rate': 0.11, 'n_estimators': 475}
CPU times: user 49.1 s, sys: 437 ms, total: 49.5 s
Wall time: 4h 8min 47s


## XGBoost (Hotel 2): Hyperparameter Tuning with Grid Search

In [9]:
estimator = XGBClassifier(objective='binary:logistic',
                          use_label_encoder=False,
                          eval_metric='logloss',
                          random_state=42,
)

# further tuning params
params = {
    'learning_rate': [0.08, 0.09, 0.1, 0.11, 0.12],
    'n_estimators': [440, 450, 475, 500, 550]
}


grid_search_2 = GridSearchCV(
    estimator = estimator,
    param_grid = params,
    n_jobs=-1,
    verbose=100
)

In [10]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2,random_state=42)

In [11]:
%%time
grid_xgb_fit_2 = grid_search_2.fit(X2_train, y2_train)
print("The best parameters are: \n", grid_xgb_fit_2.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
The best parameters are: 
 {'learning_rate': 0.12, 'n_estimators': 550}
CPU times: user 2min 8s, sys: 187 ms, total: 2min 8s
Wall time: 5h 5min 44s


### First grid search results

**H1 grid search setup**
```
estimator = XGBClassifier(objective='binary:logistic',
                          use_label_encoder=False,
                          eval_metric='logloss',
                          random_state=42,
)

params = {
    'learning_rate': [0.0001,0.01],
    'max_depth': range(2,8,2),
    'n_estimators': [200, 300, 400]
}

grid_search_1 = GridSearchCV(
    estimator = estimator,
    param_grid = params,
    n_jobs=-1,
    verbose=100
)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2,random_state=42)

```

**And the results**:
```
Fitting 5 folds for each of 18 candidates, totalling 90 fits
The best parameters are: 
 {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 400}
CPU times: user 53.5 s, sys: 414 ms, total: 53.9 s
Wall time: 1h 27min 17s
```

R^2 score: 0.830629056415377

**H2 grid search setup was the same as H1 grid search setup (round 1)**

**And the results**:

```
Fitting 5 folds for each of 18 candidates, totalling 90 fits
The best parameters are: 
 {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 400}
CPU times: user 1min 37s, sys: 262 ms, total: 1min 38s
Wall time: 1h 32min 4s
```

R^2 score: 0.8276818353712341

### Second Round

H1 setup & results
```
estimator = XGBClassifier(objective='binary:logistic',
                          use_label_encoder=False,
                          eval_metric='logloss',
                          random_state=42,
)

params = {
    'learning_rate': [0.005, 0.01, 0.05, 0.1],
    'max_depth': range(5,6,7),
    'n_estimators': [350, 400, 450]
}

grid_search_1 = GridSearchCV(
    estimator = estimator,
    param_grid = params,
    n_jobs=-1,
    verbose=100
)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2,random_state=42)

# ---------RESULTS----------
Fitting 5 folds for each of 12 candidates, totalling 60 fits
The best parameters are: 
 {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 400}
CPU times: user 40.9 s, sys: 325 ms, total: 41.2 s
Wall time: 1h 37min 13s
```

R^2 score: 0.8514727908137794

H2 results (setup the same as H1 round 2):

```

# SETUP SAME AS H1 ROUND 2

# ---------RESULTS----------
Fitting 5 folds for each of 12 candidates, totalling 60 fits
The best parameters are: 
 {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 450}
CPU times: user 1min 29s, sys: 236 ms, total: 1min 29s
Wall time: 1h 41min 45s
```
R^2 score: 0.84904827933946

### Round 3

H1 setup different than H2 setup this time.

H1 setup:
```
estimator = XGBClassifier(objective='binary:logistic',
                          use_label_encoder=False,
                          eval_metric='logloss',
                          random_state=42,
                          max_depth=5
)

params = {
    'learning_rate': [0.08, 0.09, 0.1, 0.11, 0.12],
    'n_estimators': [380, 390, 400, 410, 420, 475]
}

grid_search_1 = GridSearchCV(
    estimator = estimator,
    param_grid = params,
    n_jobs=-1,
    verbose=100
)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2,random_state=42)

# ---------RESULTS----------
Fitting 5 folds for each of 30 candidates, totalling 150 fits
The best parameters are: 
 {'learning_rate': 0.11, 'n_estimators': 475}
CPU times: user 49.1 s, sys: 437 ms, total: 49.5 s
Wall time: 4h 8min 47s
```
R^2 score: 0.854218671992012

And H2 setup:

```
estimator = XGBClassifier(objective='binary:logistic',
                          use_label_encoder=False,
                          eval_metric='logloss',
                          random_state=42,
)

# further tuning params
params = {
    'learning_rate': [0.08, 0.09, 0.1, 0.11, 0.12],
    'n_estimators': [440, 450, 475, 500, 550]
}


grid_search_2 = GridSearchCV(
    estimator = estimator,
    param_grid = params,
    n_jobs=-1,
    verbose=100
)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2,random_state=42)

# ---------RESULTS----------
Fitting 5 folds for each of 25 candidates, totalling 125 fits
The best parameters are: 
 {'learning_rate': 0.12, 'n_estimators': 550}
CPU times: user 2min 8s, sys: 187 ms, total: 2min 8s
Wall time: 5h 5min 44s
```

R^2 score: 0.8564225387621328

## XGB Scores

In [12]:
grid_xgb1_score = grid_xgb_fit_1.score(X1_test, y1_test)
grid_xgb1_score

0.854218671992012

In [13]:
grid_xgb2_score = grid_xgb_fit_2.score(X2_test, y2_test)
grid_xgb2_score

0.8564225387621328

In [14]:
grid_xgb_fit_1.to_pickle("pickle/h1_grid_result_1.pick")
grid_xgb_fit_2.to_pickle("pickle/h2_grid_result_1.pick")

AttributeError: 'GridSearchCV' object has no attribute 'to_pickle'